In [8]:
import os
import sys
import pickle
import torch
sys.path.append(os.pardir)

In [9]:
with open('../model/trained_model.pickle', mode='rb') as f:
    model = pickle.load(f)
    model.eval()

with open('../model/word_to_ix.pickle', mode='rb') as f:
    word_to_ix = pickle.load(f)

In [10]:
def text_to_id(text):
    text = text.replace('。', ' ')
    text = [word_to_ix[s] for s in list(text)]
    return torch.tensor(text, dtype=torch.long)

In [11]:
new_text = "ギャルの特性を活かしたマーケティングを行う会社「シホ有限会社G-Revo」を2005年4月28日に設立した"

In [13]:
result = model.forward(text_to_id(new_text))
for text, label in zip(list(new_text), result[1]):
    print(f'{text}: {label}')

ギ: 1
ャ: 0
ル: 0
の: 0
特: 0
性: 0
を: 0
活: 0
か: 0
し: 0
た: 0
マ: 0
ー: 0
ケ: 1
テ: 0
ィ: 1
ン: 0
グ: 0
を: 1
行: 0
う: 0
会: 0
社: 0
「: 0
シ: 1
ホ: 1
有: 0
限: 1
会: 0
社: 0
G: 1
-: 0
R: 1
e: 0
v: 1
o: 0
」: 0
を: 0
2: 0
0: 0
0: 0
5: 0
年: 0
4: 0
月: 0
2: 0
8: 0
日: 1
に: 1
設: 0
立: 0
し: 0
た: 0
